<a href="https://colab.research.google.com/github/iskra3138/colab_seminar/blob/master/autogluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
## For P100
!nvidia-smi

# AutoGluon 설치

In [0]:
!pip install --upgrade mxnet-cu100
!pip install autogluon

RESTART RUNTIME 버튼을 누르지 말고 아래 에러 해결을 위한 code cell 하나 더 실행하고 RESTART

- task.fit 실행시 'ValueError: max() arg is an empty sequence' 발생 시 아래 3가지 셀 실행
  - 출처 :<https://github.com/awslabs/autogluon/issues/163>

In [0]:
!pip uninstall -y distributed
!pip install distributed
!pip install -U ipykernel

- RESTART RUNTIME 버튼을 눌러서 런타임 다시 시작하고 아래 부터 실행

# Image Classification

[출처] <https://autogluon.mxnet.io/tutorials/image_classification/beginner.html>

In this quick start, we’ll use the task of image classification to illustrate how to use AutoGluon’s APIs.

In this tutorial, we load images and the corresponding labels into AutoGluon and use this data to obtain a neural network that can classify new images. This is different from traditional machine learning where we need to manually define the neural network and then specify the hyperparameters in the training process. Instead, with just a single call to AutoGluon’s [fit](https://autogluon.mxnet.io/api/autogluon.task.html#autogluon.task.ImageClassification.fit) function, AutoGluon automatically trains many models with different hyperparameter configurations and returns the model that achieved the highest level of accuracy.

We begin by specifying [ImageClassification](https://autogluon.mxnet.io/api/autogluon.task.html#autogluon.task.ImageClassification) as our task of interest as follows:

### Image Classification - Quick Start

In [0]:
import autogluon as ag
from autogluon import ImageClassification as task

### Create AutoGluon Dataset

For demonstration purposes, we use a subset of the [Shopee-IET dataset](https://www.kaggle.com/c/shopee-iet-machine-learning-competition/data) from Kaggle. Each image in this data depicts a clothing item and the corresponding label specifies its clothing category. Our subset of the data contains the following possible labels: BabyPants, BabyShirt, womencasualshoes, womenchiffontop.

We download the data subset and unzip it using the following commands:

In [0]:
filename = ag.download('https://autogluon.s3.amazonaws.com/datasets/shopee-iet.zip')
ag.unzip(filename)

After the dataset is downloaded, we load it into a `Dataset </api/autogluon.task.html#autogluon.task.ImageClassification.Dataset>`__ object:

In [0]:
dataset = task.Dataset('data/train')

Load the test dataset as follows:

In [0]:
test_dataset = task.Dataset('data/test', train=False)

If you don’t have a GPU, change the dataset to ‘FashionMNIST’ to ensure that it doesn’t take too long to run:

In [0]:
if ag.get_gpu_count() == 0:
    dataset = task.Dataset(name='FashionMNIST')
    test_dataset = task.Dataset(name='FashionMNIST', train=False)

### Use AutoGluon to Fit Models

Now, we fit a classifier using AutoGluon as follows:

In [0]:
classifier = task.fit(dataset,
                      epochs=10,
                      ngpus_per_trial=1,
                      verbose=False)

Within fit, the dataset is automatically split into training and validation sets. The model with the best hyperparameter configuration is selected based on its performance on the validation set. The best model is finally retrained on our entire dataset (i.e., merging training+validation) using the best configuration.

The best Top-1 accuracy achieved on the validation set is as follows:

In [0]:
print('Top-1 val acc: %.3f' % classifier.results['best_reward'])

### Predict on a New Image

Given an example image, we can easily use the final model to predict the label (and the conditional class-probability):

In [0]:
# skip this if training FashionMNIST on CPU.
if ag.get_gpu_count() > 0:
    image = 'data/test/BabyShirt/BabyShirt_323.jpg'
    #ind, prob, _ = classifier.predict(image) ## return이 2개만 된다고 해서 아래로 수정
    ind, prob = classifier.predict(image)

    print('The input picture is classified as [%s], with probability %.2f.' %
          (dataset.init().classes[ind.asscalar()], prob.asscalar()))

### Evaluate on Test Dataset

We now evaluate the classifier on a test dataset.

The validation and test top-1 accuracy are:

In [0]:
test_acc = classifier.evaluate(test_dataset)
print('Top-1 test acc: %.3f' % test_acc)

# Tabular Prediction

[출처] <https://autogluon.mxnet.io/tutorials/tabular_prediction/tabular-quickstart.html>

### Predicting Columns in a Table - Quick Start

Via a simple fit() call, AutoGluon can produce highly-accurate models to predict the values in one column of a data table based on the rest of the columns’ values. Use AutoGluon with tabular data for both classification and regression problems. This tutorial demonstrates how to use AutoGluon to produce a classification model that predicts whether or not a person’s income exceeds $50,000.

To start, import autogluon and TabularPrediction module as your task:

In [0]:
import autogluon as ag
from autogluon import TabularPrediction as task

Load training data from a CSV file into an AutoGluon Dataset object. This object is essentially equivalent to a [Pandas DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) and the same methods can be applied to both.

In [0]:
train_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')
train_data = train_data.head(500) # subsample 500 data points for faster demo
print(train_data.head())

Note that we loaded data from a CSV file stored in the cloud (AWS s3 bucket), but you can you specify a local file-path instead if you have already downloaded the CSV file to your own machine (e.g., using wget). Each row in the table train_data corresponds to a single training example. In this particular dataset, each row corresponds to an individual person, and the columns contain various characteristics reported during a census.

Let’s first use these features to predict whether the person’s income exceeds $50,000 or not, which is recorded in the class column of this table.

In [0]:
label_column = 'class'
print("Summary of class variable: \n", train_data[label_column].describe())

Now use AutoGluon to train multiple models:

In [0]:
dir = 'agModels-predictClass' # specifies folder where to store trained models
predictor = task.fit(train_data=train_data, label=label_column, output_directory=dir)

Next, load separate test data to demonstrate how to make predictions on new examples at inference time:

In [0]:
test_data = task.Dataset(file_path='https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
y_test = test_data[label_column]  # values to predict
test_data_nolab = test_data.drop(labels=[label_column],axis=1) # delete label column to prove we're not cheating
print(test_data_nolab.head())

We use our trained models to make predictions on the new data and then evaluate performance:

In [0]:
predictor = task.load(dir) # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  ", y_pred)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)

Now you’re ready to try AutoGluon on your own tabular datasets!
As long as they’re stored in a popular format like CSV, you should be able to achieve strong predictive performance with just 2 lines of code:

In [0]:
from autogluon import TabularPrediction as task
predictor = task.fit(train_data=task.Dataset(file_path=<file-name>), label_column=<variable-name>)

### Description of fit():

Here we discuss what happened during fit().

Since there are only two possible values of the class variable, this was a binary classification problem, for which an appropriate performance metric is accuracy. AutoGluon automatically infers this as well as the type of each feature (i.e., which columns contain continuous numbers vs. discrete categories). AutogGluon can also automatically handle common issues like missing data and rescaling feature values.

We did not specify separate validation data and so AutoGluon automatically choses a random training/validation split of the data. The data used for validation is seperated from the training data and is used to determine the models and hyperparameter-values that produce the best results. Rather than just a single model, AutoGluon trains multiple models and ensembles them together to ensure superior predictive performance.

By default, AutoGluon tries to fit various types of models including neural networks and tree ensembles. Each type of model has various hyperparameters, which traditionally, the user would have to specify. AutoGluon automates this process.

AutoGluon automatically and iteratively tests values for hyperparameters to produce the best performance on the validation data. This involves repeatedly training models under different hyperparameter settings and evaluating their performance. This process can be computationally-intensive, so fit() can parallelize this process across multiple threads (and machines if distributed resources are available). To control runtimes, you can specify various arguments in fit() as demonstrated in the subsequent In-Depth tutorial.

For tabular problems, fit() returns a Predictor object. Besides inference, this object can also be used to view a summary of what happened during fit.

In [0]:
results = predictor.fit_summary()

From this summary, we can see that AutoGluon trained many different types of models as well as an ensemble of the best-performing models. The summary also describes the actual models that were trained during fit and how well each model performed on the held-out validation data. We can also view what properties AutoGluon automatically inferred about our prediction task:

In [0]:
print("AutoGluon infers problem type is: ", predictor.problem_type)
print("AutoGluon categorized the features as: ", predictor.feature_types)

AutoGluon correctly recognized our prediction problem to be a binary classification task and decided that variables such as age should be represented as integers, whereas variables such as workclass should be represented as categorical objects.

### Regression (predicting numeric table columns):

To demonstrate that fit() can also automatically handle regression tasks, we now try to predict the numeric age variable in the same table based on the other features:

In [0]:
age_column = 'age'
print("Summary of age variable: \n", train_data[age_column].describe())

We again call fit() and this time use a shorthand method to evaluate the resulting model on the test data (which contain labels):

In [0]:
predictor_age = task.fit(train_data=train_data, output_directory="agModels-predictAge", label=age_column)
performance = predictor_age.evaluate(test_data)

Note that we didn’t need to tell AutoGluon this is a regression problem, it automatically inferred this from the data and reported the appropriate performance metric (RMSE by default).

# Object Detection

[출처] <https://autogluon.mxnet.io/tutorials/object_detection/beginner.html>

### Object Detection - Quick Start

Object detection is the process of identifying and localizing objects in an image and is an important task in computer vision. Follow this tutorial to learn how to use AutoGluon for object detection.

Tip: If you are new to AutoGluon, review [Image Classification - Quick Start](https://autogluon.mxnet.io/tutorials/image_classification/beginner.html#sec-imgquick) first to learn the basics of the AutoGluon API.

Our goal is to detect motorbike in images by [YOLO3 model](https://pjreddie.com/media/files/papers/YOLOv3.pdf). A tiny dataset is collected from VOC dataset, which only contains the motorbike category. The model pretrained on the COCO dataset is used to fine-tune our small dataset. With the help of AutoGluon, we are able to try many models with different hyperparameters automatically, and return the best one as our final model.

To start, import autogluon and ObjectDetection module as your task:

In [0]:
import autogluon as ag
from autogluon import ObjectDetection as task

### Tiny_motorbike Dataset

We collect a toy dataset for detecting motorbikes in images. From the VOC dataset, images are randomly selected for training, validation, and testing - 120 images for training, 50 images for validation, and 50 for testing. This tiny dataset follows the same format as VOC.

Using the commands below, we can download this dataset, which is only 23M. The variable root specifies the path to store the dataset in. The name of unzipped folder is called tiny_motorbike.

In [0]:
root = './'
filename_zip = ag.download('https://autogluon.s3.amazonaws.com/datasets/tiny_motorbike.zip',
                        path=root)
filename = ag.unzip(filename_zip, root=root)

When we retrieve the dataset, we can create a dataset instance with its path and classes if it is a custom dataset.

In [0]:
import os
data_root = os.path.join(root, filename)
dataset_train = task.Dataset(data_root, classes=('motorbike',))

### Fit Models by AutoGluon

In this section, we demonstrate how to apply AutoGluon to fit our detection models. We use mobilenet as the backbone for the YOLO3 model. Two different learning rates are used to fine-tune the network. The best model is the one that obtains the best performance on the validation dataset. You can also try using more networks and hyperparameters to create a larger searching space.

We fit a classifier using AutoGluon as follows. In each experiment (one trial in our searching space), we train the model for 30 epoches.

In [0]:
time_limits = 5*60*60  # 5 hours
epochs = 30
detector = task.fit(dataset_train,
                    num_trials=2,
                    epochs=epochs,
                    lr=ag.Categorical(5e-4, 1e-4),
                    ngpus_per_trial=1,
                    time_limits=time_limits)

After fitting, AutoGluon automatically returns the best model among all models in the searching space. From the output, we know the best model is the one trained with the second learning rate. To see how well the returned model performed on test dataset, call detector.evaluate().

In [0]:
dataset_test = task.Dataset(data_root, index_file_name='test', classes=('motorbike',))

test_map = detector.evaluate(dataset_test)
print("mAP on test dataset: {}".format(test_map[1][1]))

Below, we randomly select an image from test dataset and show the predicted box and probability over the origin image.

In [0]:
image = '000467.jpg'
image_path = os.path.join(data_root, 'JPEGImages', image)

ind, prob, loc = detector.predict(image_path)

# Text Classification

[출처] <https://autogluon.mxnet.io/tutorials/text_classification/beginner.html>

### Text Classification - Quick Start

We adopt the task of Text Classification as a running example to illustrate basic usage of AutoGluon’s NLP capability.

In this tutorial, we are using sentiment analysis as a text classification example. We will load sentences and the corresponding labels (sentiment) into AutoGluon and use this data to obtain a neural network that can classify new sentences. Different from traditional machine learning where we need to manually define the neural network, and specify the hyperparameters in the training process, with just a single call to AutoGluon’s fit function, AutoGluon will automatically train many models under thousands of different hyperparameter configurations and then return the best model.

We begin by specifying TextClassification as our task of interest:

In [0]:
import autogluon as ag
from autogluon import TextClassification as task

### Create AutoGluon Dataset

We are using a subset of the Stanford Sentiment Treebank ([SST](https://nlp.stanford.edu/sentiment/)). The original dataset consists of sentences from movie reviews and human annotations of their sentiment. The task is to classify whether a given sentence has positive or negative sentiment (binary classification).

In [0]:
dataset = task.Dataset(name='ToySST')

In the above call, we have the proper train/validation/test split of the SST dataset.

### Use AutoGluon to fit Models

Now, we want to obtain a neural network classifier using AutoGluon. In the default configuration, rather than attempting to train complex models from scratch using our data, AutoGluon fine-tunes neural networks that have already been pretrained on large scale text dataset such as Wikicorpus. Although the dataset involves entirely different text, lower-level features captured in the representations of the pretrained network (such as edge/texture detectors) are likely to remain useful for our own text dataset.

While we primarily stick with default configurations in this Beginner tutorial, the Advanced tutorial covers various options that you can specify for greater control over the training process. With just a single call to AutoGluon’s fit function, AutoGluon will train many models with different hyperparameter configurations and return the best model.

However, neural network training can be quite time-costly. To ensure quick runtimes, we tell AutoGluon to obey strict limits: num_training_epochs specifies how much computational effort can be devoted to training any single network, while time_limits in seconds specifies how much time fit has to return a model. For demo purposes, we specify only small values for time_limits, num_training_epochs:

In [0]:
predictor = task.fit(dataset, epochs=1)

Within fit, the model with the best hyperparameter configuration is selected based on its validation accuracy after being trained on the data in the training split.

The best Top-1 accuracy achieved on the validation set is:

In [0]:
print('Top-1 val acc: %.3f' % predictor.results['best_reward'])

Within fit, this model is also finally fitted on our entire dataset (i.e., merging training+validation) using the same optimal hyperparameter configuration. The resulting model is considered as final model to be applied to classify new text.

We now construct a test dataset similarly as we did with the train dataset, and then evaluate the final model produced by fit on the test data:

In [0]:
test_acc = predictor.evaluate(dataset)
print('Top-1 test acc: %.3f' % test_acc)

Given an example sentence, we can easily use the final model to predict the label (and the conditional class-probability):

In [0]:
sentence = 'I feel this is awesome!'
ind = predictor.predict(sentence)
print('The input sentence sentiment is classified as [%d].' % ind.asscalar())

The results object returned by fit contains summaries describing various aspects of the training process. For example, we can inspect the best hyperparameter configuration corresponding to the final model which achieved the above (best) results:

In [0]:
print('The best configuration is:')
print(predictor.results['best_config'])